# Transformation given the destination of three points

I have a collection of pictures with graph paper on the background. The goal is to transform each image so the background grid is at scale by a sequence of rotations, rescalings along the two axes, and transformations.  
To do this I record three points $O$, $A$, and $B$ on the grid, with $A$ 1cm to the right of $O$ and $B$ 1cm below it and look for a transformation $T$ such that
$$ T (A-O) = \operatorname{e}_{1} \, , \\ T (B-O) = \operatorname{e}_{2} \, . $$
By polar decomposition, $T$ can be expressed as a product $UP$, where $U$ is a rotation and $P$ is symmetric, which in turn means by the spectral theorem that $P = R \Delta R^{*}$ for some rotation $R$ and diagonal matrix $\Delta$. Overall, we have
$$ T = U R \Delta R^{*} $$
and up to adjusting by a translation this gives the transformation we want.  
Now, the matrix $M$ obtained by juxtaposing $A-O$ and $B-O$ (as column vectors) sends the basis vectors to $A-O$ and $B-O$, respectively, so it is the *inverse* of the one representing $T$, therefore
$$ M^{*} M = \bigl(R \Delta^{-1} R^{*} U^{*}\bigr)^{*} R \Delta^{-1} R^{*} U^{*} = U R \Delta^{-2} R^{*} U^{*} \, . $$
This promptly gives us the eivenvalues of $\Delta$ by factorising the characteristic polynomial of $M^{*} M$, and therefore $\Delta$ itself.  
Moreover, if $\lambda$ is either one of these eigenvalues, then $M^{*} M - \lambda^{2} \operatorname{I}$ is a matrix of rank $1$ whose kernel and column space are spanned by the $\lambda$-eigenvector of $M^{*} M$ and its orthogonal, respectively. Since the eigenvectors of $M^{*} M$ relative to distinct eigenvalues are orthogonal, this means that the first column of $M^{*}M$ is the other eigenvector.  
Notice now that $R^{*} U^{*}$ is a rotation that takes the $\lambda$-eigenvector of $M^{*} M$ to that of $\Delta^{-2}$, and similarly for the orthogonal. Therefore, the angle by which $UR$ rotates is the angle from $\operatorname{e}_{2}$ to the first column of $M^{*} M - \lambda \operatorname{I}$. This gives us $UR$.  
Finally, since $M = R \Delta^{-1} R^{*} U^{*}$ and $M \operatorname{e}_{2} = B-O$, we can conclude that $R$ is the rotation matrix that transforms $\Delta^{-1} R^{*} U^{*} \operatorname{e}_{2}$ into $B-O$. On the other hand, since $UR$ transforms $\operatorname{e}_{2}$ into the first column of $M^{*} M$, the image $R^{*} U^{*} \operatorname{e}_{2}$ under the inverse transformation will be the same but reflected around the $y$-axis. Finding the angle is then a matter of taking vector product of this object with $B-O$.

In [209]:
from math import sqrt

class Matrix:
    def __init__(self, entries: list[float]=[0, 0, 0, 0]):
        self.entries = entries
    
    def __repr__(self) -> str:
        return f'Matrix({self.entries})'
    
    def __str__(self) -> str:
        return f'[{self.entries[:2]},\n {self.entries[2:]}]'
    
    def __getitem__(self, idx: tuple[int]) -> float:
        return self.entries[2*idx[0] + idx[1] - 3]
    
    def __setitem__(self, idx: tuple[int], val: float) -> None:
        self.entries[2*idx[0] + idx[1] - 3] = val
    
    def transposed(self) -> 'Matrix':
        return Matrix([self.entries[0], self.entries[2], self.entries[1], self.entries[3]])
    
    def __mul__(self, other: 'Matrix') -> 'Matrix':
        product = Matrix()
        for i in [1, 2]:
            for j in [1, 2]:
                product[(i,j)] = sum([self[(i,k)]*other[(k,j)] for k in [1, 2]])
        return product

    def __rmul__(self, other: float) -> 'Matrix':
        return Matrix([other*entry for entry in self.entries])
    
    def __add__(self, other: 'Matrix') -> 'Matrix':
        new_matrix = Matrix()
        print(new_matrix, self, other)
        for i in [1, 2]:
            for j in [1, 2]:
                # print('Before: ', i, j, new_matrix[(i,j)], self[(i,j)], other[(i,j)])
                new_matrix[(i, j)] = self[(i, j)] + other[(i, j)]
                # print(new_matrix[(i,j)], self[(i,j)], other[(i,j)])
        return new_matrix
    
    def __neg__(self) -> 'Matrix':
        return Matrix([-x for x in self.entries])
    
    def __sub__(self, other: 'Matrix') -> 'Matrix':
        return self + (-other)
    
    def trace(self) -> float:
        return self[(1,1)] + self[(2,2)]

    def det(self) -> float:
        return self[(1,1)]*self[(2,2)] - self[(1,2)]*self[(2,1)]
    
    def eigenvals(self):
        Delta = self.trace()**2 - 4*self.det()
        if Delta < 0:
            raise ValueError('The matrix has no real eigenvalues.')
        return (self.trace()-sqrt(Delta))/2, (self.trace()+sqrt(Delta))/2
    
IdMatrix = Matrix([1, 0, 0, 1])

def diag(eigenvals: tuple[float]):
    return Matrix([eigenvals[0], 0, 0, eigenvals[1]])

In [210]:
from math import asin as arcsin
from math import pi

def figure_it_out(O: list[float], A: list[float], B: list[float]):
    M = Matrix([A[0] - O[0], B[0] - O[0], A[1] - O[1], B[1] - O[1]])
    print(f'M: {M}')
    
    MtM = M.transposed()*M
    print(f'MtM: {MtM}')
    eigenvals = MtM.eigenvals()
    print('Check eigenvalues', [MtM-v*IdMatrix for v in MtM.eigenvals()])
    Delta = diag([l**(-.5) for l in eigenvals])
    print(f'Delta:\n{Delta}')
    print(MtM.det()*Delta.det()**2)

    eigenvects = MtM - eigenvals[0]*IdMatrix
    vect = [eigenvects[(1,1)], eigenvects[(2,1)]]
    if vect[0] == 0 and vect[1] == 0:
        vect = [eigenvects[(1,2)], eigenvects[(2,2)]]
    if vect[0] == 0 and vect[1] == 0:
        vect = [0, 1]
    URsine = -vect[1]/sqrt(vect[0]**2 + vect[1]**2)
    print(f'Angle of rotation of UR: {arcsin(URsine)*180/pi}')

    N = Matrix([B[0]-O[0], -sqrt(eigenvals[0])*vect[0], B[1]-B[0], sqrt(eigenvals[1]*vect[1])])
    Rinvsine = N.det()/sqrt((N.transposed()*N)[(1,1)] * (N.transposed()*N)[(2,2)])
    print(f'Angle of rotation of R inverse: {arcsin(Rinvsine)*180/pi}')
    

In [211]:
figure_it_out(O = [-1, -2], A = [2, 1], B = [0, 0])

M: [[3, 1],
 [3, 2]]
MtM: [[18, 9],
 [9, 5]]
[[18, 9],
 [9, 5]] [[18, 9],
 [9, 5]] [[-0.3981983444127408, -0.0],
 [-0.0, -0.3981983444127408]]
[[17.60180165558726, 9.0],
 [9.0, 4.601801655587259]] [[17.60180165558726, 9.0],
 [9.0, 4.601801655587259]] [[-22.60180165558726, -0.0],
 [-0.0, -22.60180165558726]]
Check eigenvalues [Matrix([-5.0, 9.0, 9.0, -18.0]), Matrix([-5.0, 9.0, 9.0, -18.0])]
Delta:
[[1.5847117387920282, 0],
 [0, 0.21034319691947392]]
1.000000000000001
[[-5.0, 9.0],
 [9.0, -18.0]] [[-5.0, 9.0],
 [9.0, -18.0]] [[-0.3981983444127408, -0.0],
 [-0.0, -0.3981983444127408]]
Angle of rotation of UR: -59.044677812535205
Angle of rotation of R inverse: 76.5671648521434


In [113]:
from math import sin, cos
r = -72.24782024350702*pi/180
R = Matrix([cos(r), -sin(r), sin(r), cos(r)])
ruinv = 31.44593856459911*pi/180
RUinv = Matrix([cos(ruinv), -sin(ruinv), sin(ruinv), cos(ruinv)])
Deltainv = Matrix([1.0981643143898987**(-1), 0, 0, 0.20235789791228198**(-1)])
print(RUinv.transposed()*Deltainv*Deltainv*RUinv)


[[7.25, 7.086780371577321],
 [10.5, 12.296532948813056]]


In [176]:
A = Matrix([9,.7, 0, -8.1])
B = Matrix([5, 6, 0, 6])
A[(2,1)] = 6
print(A)

[[9, 0.7],
 [6, -8.1]]
